___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Авторские права принадлежат Pierian Data Inc.</em></center>
<center><em>Для дополнительной информации посетите наш сайт <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Извлечение признаков из текста

Этот блокнот разделён на две части:
* Сначала мы выясним, что нам понадобится для построения инструментария NLP (Natural Language Processing), который превратит набор текста в числовой массив из *признаков*. Для этого мы вручную вычислим, как часто встречаются те или иные слова, и построим метрику TF-IDF.
* Далее мы выполним эти шаги с помощью scikit-learn.

# Часть 1: основные принципы извлечения признаков из текста


В этой части мы с помощью базовых операций Python построим очень простую систему NLP. Мы возьмём *набор документов* (*corpus of documents*) - это будет два текстовых файла. Далее создадим *словарь* (*vocabulary*) из всех слов обоих документов. И затем посмотрим на технику *Мешок слов* (*Bag of Words*) для извлечения признаков из каждого документа.<br>
<div class="alert alert-info" style="margin: 20px">В этом разделе мы только приводим иллюстрации основных принципов!
<br>Не обращайте внимание на то, как пишется код в Python - позднее мы применим для этой задачи Scikit-Learn.</div>

## Начнём с документов:
Для простоты изложения в наших текстовых файлах One.txt и Two.txt не будет каких-либо знаков пунктуации. Давайте откроем эти файлы и прочитаем данные. Обратите внимание, что если в будущем у Вас файлы будут большие, то их не следует выводить на экран полностью.


In [1]:
with open('One.txt') as mytext:
    print(mytext.read())

This is a story about dogs
our canine pets
Dogs are furry animals



In [2]:
with open('Two.txt') as mytext:
    print(mytext.read())

This story is about surfing
Catching waves is fun
Surfing is a popular water sport



### Читаем весь текст как строку string

In [3]:
with open('One.txt') as mytext:
    entire_text = mytext.read()

In [4]:
entire_text

'This is a story about dogs\nour canine pets\nDogs are furry animals\n'

In [5]:
print(entire_text)

This is a story about dogs
our canine pets
Dogs are furry animals



### Читаем каждую строку файла отдельно и помещаем в список

In [6]:
with open('One.txt') as mytext:
    lines = mytext.readlines()

In [7]:
lines

['This is a story about dogs\n',
 'our canine pets\n',
 'Dogs are furry animals\n']

### Читаем отдельные слова

In [8]:
with open('One.txt') as f:
    words = f.read().lower().split()

In [9]:
words

['this',
 'is',
 'a',
 'story',
 'about',
 'dogs',
 'our',
 'canine',
 'pets',
 'dogs',
 'are',
 'furry',
 'animals']

## Создаём словарь vocabulary (Мешок слов - "Bag of Words")

Для этого мы возьмём все слова из обоих документов, найдём уникальные слова, и пронумеруем их.

In [10]:
with open('One.txt') as f:
    words_one = f.read().lower().split()

In [11]:
words_one

['this',
 'is',
 'a',
 'story',
 'about',
 'dogs',
 'our',
 'canine',
 'pets',
 'dogs',
 'are',
 'furry',
 'animals']

In [12]:
len(words_one)

13

In [13]:
uni_words_one = set(words)

In [14]:
uni_words_one

{'a',
 'about',
 'animals',
 'are',
 'canine',
 'dogs',
 'furry',
 'is',
 'our',
 'pets',
 'story',
 'this'}

**То же самое для файла Two.txt**

In [15]:
with open('Two.txt') as f:
    words_two = f.read().lower().split()
    uni_words_two = set(words_two)

In [16]:
uni_words_two

{'a',
 'about',
 'catching',
 'fun',
 'is',
 'popular',
 'sport',
 'story',
 'surfing',
 'this',
 'water',
 'waves'}

**Получаем все уникальные слова из всех документов**

In [17]:
all_uni_words = set()
all_uni_words.update(uni_words_one)
all_uni_words.update(uni_words_two)

In [18]:
all_uni_words

{'a',
 'about',
 'animals',
 'are',
 'canine',
 'catching',
 'dogs',
 'fun',
 'furry',
 'is',
 'our',
 'pets',
 'popular',
 'sport',
 'story',
 'surfing',
 'this',
 'water',
 'waves'}

In [19]:
full_vocab = dict()
i = 0

for word in all_uni_words:
    full_vocab[word] = i
    i = i+1

In [20]:
# Эти слова могут быть НЕ в алфавитном порядке! 
# Цикл for вовсе НЕ обязан проходит по множеству set() в алфавитном порядке!
full_vocab

{'waves': 0,
 'surfing': 1,
 'catching': 2,
 'dogs': 3,
 'fun': 4,
 'this': 5,
 'pets': 6,
 'sport': 7,
 'popular': 8,
 'story': 9,
 'our': 10,
 'a': 11,
 'water': 12,
 'animals': 13,
 'furry': 14,
 'canine': 15,
 'about': 16,
 'are': 17,
 'is': 18}

## Мешок слов - считаем, как часто встречаются отдельные слова

После того, как мы создали словарь из слов, давайте выполним *извлечение признаков* для каждого из двух исходных документов:

**Создаём пустые счётчики для каждого документа**

In [21]:
# Для каждого документа создаём пустые счётчики для всех слов из словаря full_vocab:
one_freq = [0]*len(full_vocab)
two_freq = [0]*len(full_vocab)
all_words = ['']*len(full_vocab)

In [22]:
one_freq

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [23]:
two_freq

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [24]:
all_words

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

In [25]:
for word in full_vocab:
    word_ind = full_vocab[word]
    all_words[word_ind] = word    

In [26]:
all_words

['waves',
 'surfing',
 'catching',
 'dogs',
 'fun',
 'this',
 'pets',
 'sport',
 'popular',
 'story',
 'our',
 'a',
 'water',
 'animals',
 'furry',
 'canine',
 'about',
 'are',
 'is']

**Добавляем счётчики слов для каждого документа:**

In [27]:
# Берём слова из файла, для каждого слова ищем соответствие в нашем словаре full_vocab:
with open('One.txt') as f:
    one_text = f.read().lower().split()
    
for word in one_text:
    word_ind = full_vocab[word]
    one_freq[word_ind]+=1

In [28]:
one_freq

[0, 0, 0, 2, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]

In [29]:
# То же самое для второго документа:
with open('Two.txt') as f:
    two_text = f.read().lower().split()
    
for word in two_text:
    word_ind = full_vocab[word]
    two_freq[word_ind]+=1

In [30]:
two_freq

[1, 2, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 3]

In [31]:
import pandas as pd
pd.DataFrame(data=[one_freq,two_freq],columns=all_words)

,waves,surfing,catching,dogs,fun,this,pets,sport,popular,story,our,a,water,animals,furry,canine,about,are,is
0,0,0,0,2,0,1,1,0,0,1,1,1,0,1,1,1,1,1,1
1,1,2,1,0,1,1,0,1,1,1,0,1,1,0,0,0,1,0,3


Некоторые слова есть в обоих документах, а некоторые слова есть только в `One.txt`, и некоторые другие слова есть только в `Two.txt`. Если мы применим эту логику для десятков тысяч документов, то наш словарь вполне сможет вырасти до десятков тысяч слов. При этом матрицы будут содержать очень много нулей - это будут **разреженные метрицы** (**sparse matrices**).


# Продолжение:

## Мешок слов и Tf-idf
В приведённом выше примере, каждый вектор можно рассматривать как мешок слов (*bag of words*). Сами по себе эти вектора не очень полезны, но мы также можем добавить к ним частоту слов (*term frequencies - TF*) - как часто то или иное слово встречается в документе. Простой способ сделать это - это посчитать, сколько раз встречается слово в документе, и разделить на общее количество слов в документе. Тогда можно сравнивать между большими и маленькими документами, сколько раз то или иное слово встречается в документе.

Однако, если какое-то слово встречается во многих документах, то такое слово не будет являться хорошим признаком для отделения документов друг от друга. Чтобы работать с такими словами, можно добавить метрику *inverse document frequency (IDF)*, которая вычисляется как общее количество документов, разделить на количество документов, в которых содержится рассматриваемое нами слово (слово, для которого вычисляется IDF). В практических задачах это значение приводится к логарифмической шкале, подробнее см. [здесь](https://ru.wikipedia.org/wiki/TF-IDF).

Соединяя метрики TF (Term Frequency) и IDF (Inverse Document Frequency), мы получаем метрику [**tf-idf**](https://ru.wikipedia.org/wiki/TF-IDF).

## Стоп-слова и морфология слов
Некоторые слова встречаются слишком часто, например слова "the" и "and" в английском языке. Эти слова можно просто исключить из рассмотрения. 

Кроме этого, одно и то же слово может встречаться в единственном и множественном числе, а также в различных падежах. Нам бы хотелось записать это слово один раз, а не записывать различные его вариации (например, записать только `cat` для обоих слов `cat` и `cats`). Это позволит нам уменьшить размер словаря и увеличить скорость работы модели.

С другой стороны, различные падежи слов - особенно в русском языке - несут дополнительную смысловую информацию. Эту информацию по-хорошему можно сохранять, то есть ссылаться на базовое слово в словаре, плюс сохранять дополнительные атрибуты этого слова в конкретном месте текста. В нашем блокноте мы ограничимся лишь базовым подсчётом частоты слов - как в отдельном документе, так и в целом наборе документов.

# Часть 2: извлечение признаков и текста в Scikit-Learn

Давайте применим sklearn для решения этих задач!

# Варианты извлечения признаков в Scikit-Learn

In [32]:
text = ['This is a line',
           "This is another line",
       "Completely different line"]

## CountVectorizer

In [33]:
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer,CountVectorizer

In [34]:
cv = CountVectorizer()

In [35]:
cv.fit_transform(text)

<3x6 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [36]:
sparse_mat = cv.fit_transform(text)

In [37]:
sparse_mat.todense()

matrix([[0, 0, 0, 1, 1, 1],
        [1, 0, 0, 1, 1, 1],
        [0, 1, 1, 0, 1, 0]])

In [38]:
cv.vocabulary_

{'this': 5, 'is': 3, 'line': 4, 'another': 0, 'completely': 1, 'different': 2}

In [39]:
cv = CountVectorizer(stop_words='english')

In [40]:
cv.fit_transform(text).todense()

matrix([[0, 0, 1],
        [0, 0, 1],
        [1, 1, 1]])

In [41]:
cv.vocabulary_

{'line': 2, 'completely': 0, 'different': 1}

## TfidfTransformer

TfidfVectorizer применяется для текстовых документов, а TfidfTransformer применяется к матрице со счётчиками, которую возвращает CountVectorizer

In [42]:
tfidf_transformer = TfidfTransformer()

In [43]:
cv = CountVectorizer()

In [44]:
counts = cv.fit_transform(text)

In [45]:
counts

<3x6 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [46]:
tfidf = tfidf_transformer.fit_transform(counts)

In [47]:
tfidf.todense()

matrix([[0.        , 0.        , 0.        , 0.61980538, 0.48133417,
         0.61980538],
        [0.63174505, 0.        , 0.        , 0.4804584 , 0.37311881,
         0.4804584 ],
        [0.        , 0.65249088, 0.65249088, 0.        , 0.38537163,
         0.        ]])

In [48]:
from sklearn.pipeline import Pipeline

In [49]:
pipe = Pipeline([('cv',CountVectorizer()),('tfidf',TfidfTransformer())])

In [50]:
results = pipe.fit_transform(text)

In [51]:
results

<3x6 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [52]:
results.todense()

matrix([[0.        , 0.        , 0.        , 0.61980538, 0.48133417,
         0.61980538],
        [0.63174505, 0.        , 0.        , 0.4804584 , 0.37311881,
         0.4804584 ],
        [0.        , 0.65249088, 0.65249088, 0.        , 0.38537163,
         0.        ]])

## TfIdfVectorizer

Выполняем оба действия единым шагом!

In [53]:
tfidf = TfidfVectorizer()

In [54]:
new = tfidf.fit_transform(text)

In [55]:
new.todense()

matrix([[0.        , 0.        , 0.        , 0.61980538, 0.48133417,
         0.61980538],
        [0.63174505, 0.        , 0.        , 0.4804584 , 0.37311881,
         0.4804584 ],
        [0.        , 0.65249088, 0.65249088, 0.        , 0.38537163,
         0.        ]])